## Load Libraries

In [ ]:
# pip install torch torchvision torchaudio
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Define Transformations

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

## Load the Data

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_data = datasets.CIFAR10( root="./data", train=True, download=True, transform = transform)
test_data = datasets.CIFAR10( root="./data", train=False, download=True, transform = transform)

In [4]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

## Build the Architecture

In [24]:
class cnn_cifar(nn.Module):
    def __init__(self):
        super(cnn_cifar, self).__init__()
        self.cnn1 = nn.Conv2d(3, 6, 5)
        self.cnn2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x=self.cnn1(x)
        x=torch.relu(x)
        x=torch.max_pool2d(x, 2)

        x=self.cnn2(x)
        x=torch.relu(x)
        x=torch.max_pool2d(x, 2)

        x=x.flatten(1)

        x=self.fc1(x)
        x=torch.relu(x)

        x=self.fc2(x)
        x=torch.relu(x)

        x=self.fc3(x)

        return torch.relu(x)

In [26]:
model = cnn_cifar()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

##  Training

In [30]:
for epoch in range(10):
    for images, label in train_loader:
        outputs = model(images)
        loss = criterion(outputs, label)
        optimizer.step()
        optimizer.zero_grad()
    print(f"epoch : {epoch+1}/10 ---- loss : {loss.item():.4f}")
        

epoch : 1/10 ---- loss : 2.3240
epoch : 2/10 ---- loss : 2.2965
epoch : 3/10 ---- loss : 2.2997
epoch : 4/10 ---- loss : 2.3030
epoch : 5/10 ---- loss : 2.2951
epoch : 6/10 ---- loss : 2.3257
epoch : 7/10 ---- loss : 2.2953
epoch : 8/10 ---- loss : 2.3057
epoch : 9/10 ---- loss : 2.2819
epoch : 10/10 ---- loss : 2.3142


In [38]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs=model(images)
        _, pred = torch.max(outputs, 1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)
    print(f"Test Accuracy : {100*correct/total:.2f}%")

Test Accuracy : 10.00%
